In [1]:
import numpy as np
import pandas as pd
from scipy import sparse

from bmfm_targets.datasets.SNPdb.tabix_converter import (
    extract_chr_seq_and_len,
    sample_variant,
)

np.set_printoptions(precision=3,suppress=True)

/dccstor/bmfm-targets/users/sanjoy/targets_env/anaconda/lib/python3.10/site-packages/pandera/engines/pandas_engine.py:66: UserWarning: Using typeguard < 3. Generic types like List[TYPE], Dict[TYPE, TYPE] will only validate the first element in the collection.
  warnings.warn(


In [2]:
def extract_subsequence(sequence, left, right, len_total=None):
    if len_total:
        center = (left + right) // 2
        left = center - len_total // 2
        right = left + len_total
    return sequence[left:right]

def count_variation_percent(sequence):
    return sum([x not in 'ACGTN' for x in sequence]) / len(sequence)


In [3]:
path0 = '/dccstor/bmfm-targets/data/omics/genome/snpdb/raw/matrices/'
SNPDB_RESOURCES_PATH = "/dccstor/bmfm-targets/data/omics/genome/snpdb/raw/resources/"
nucleotide_lexicon_path = SNPDB_RESOURCES_PATH + "nucleotide_lexicon.json"
biallele_lexicon_path = SNPDB_RESOURCES_PATH + "biallele_lexicon.json"
fasta_path = SNPDB_RESOURCES_PATH + "hs37d5.fa"

input_file = '/dccstor/bmfm-targets/data/omics/genome/MPGA/human_mpra/K562_clean_outfmt6processed.out'
label_file = '/dccstor/bmfm-targets/data/omics/genome/MPGA/human_mpra/K562_clean.tsv'
output1_path = '/dccstor/bmfm-targets/data/omics/genome/MPGA/human_mpra/K562_biallele_sequence_200.csv'
output2_path = '/dccstor/bmfm-targets/data/omics/genome/MPGA/human_mpra/K562_biallele_sequence_20kb.csv'
output1_ref_path = output1_path.replace("biallele", "ref")
output2_ref_path = output2_path.replace("biallele", "ref")

### There are special "chromosomes"

In [4]:
df0 = pd.read_csv(input_file,sep='\t')
print(df0.shape, df0.head(2))

df_label = pd.read_csv(label_file,sep='\t')
print(df_label.shape, df_label.head(2))

print("retriving the labels...")
df0 = df0.merge(df_label, how='left', left_on='qseqid', right_on='seq_id')

print(df0.head(2), df0.shape)
print(df_label.loc[df_label['seq_id']=='BCL11A_1', :])

(225930, 6)       qseqid sseqid  qstart  qend    sstart      send
0   BCL11A_1      2     145   175  25588744  25588774
1  BCL11A_10      2      15   215  59782433  59782633
(226253, 4)              seq_id                                                seq  \
0            peak10  AGGACCGGATCAACTTGTCGCCTTAATCCAAGCCTACGTTTTTACA...   
1  peak10_Reversed:  AGGACCGGATCAACTAGTATGAGGAGGGTTGTGGAGTGGAAGTGAA...   

   mean_value  fold  
0      -0.490     6  
1      -0.131     6  
retriving the labels...
      qseqid sseqid  qstart  qend    sstart      send     seq_id  \
0   BCL11A_1      2     145   175  25588744  25588774   BCL11A_1   
1  BCL11A_10      2      15   215  59782433  59782633  BCL11A_10   

                                                 seq  mean_value  fold  
0  AGGACCGGATCAACTTAAATGACTTCAACTGCCCCAACCCCTCTTC...      -0.475     4  
1  AGGACCGGATCAACTACATTTGCTGAGGAGAGCTTTACTTCCAACT...      -0.517     4   (225930, 10)
          seq_id                                                

In [5]:
df0.sseqid.value_counts()

sseqid
1             22372
2             19182
11            18154
6             13433
8             12474
3             11968
7             11130
16            10749
X              9769
12             9668
20             9350
5              9141
17             9123
10             9106
4              8569
19             8127
9              7554
15             6343
14             5237
22             4258
18             3438
13             3415
21             2670
hs37d5          500
Y                56
GL000193.1       16
GL000192.1       13
GL000191.1       10
GL000223.1       10
GL000202.1        8
GL000204.1        8
GL000249.1        8
GL000209.1        6
GL000197.1        6
GL000227.1        6
GL000226.1        6
GL000205.1        6
GL000201.1        6
GL000242.1        4
GL000248.1        4
GL000222.1        4
GL000210.1        4
GL000198.1        4
GL000211.1        4
GL000241.1        2
GL000246.1        2
GL000212.1        2
GL000196.1        2
GL000200.1        2
GL000219.1   

### Extract the reference genome first

In [15]:
# Create the ref genome
chr_to_seq, _ = extract_chr_seq_and_len(fasta_path, ">")

with open(output1_ref_path, "w") as ref_f1, open(output2_ref_path, "w") as ref_f2:
    ref_f1.write("seq_id,chunk,mean_value,fold\n")
    ref_f2.write("seq_id,chunk,mean_value,fold\n")
    for i in range(1,23):
        target_chr = 'chr' + str(i)
        the_df = df0[df0.sseqid == str(i)]
        print('chr' + str(i), len(the_df))
        ref_seq = chr_to_seq[target_chr]

        for j in range(len(the_df)):
            the_id = the_df['qseqid'].iloc[j]
            label = the_df['mean_value'].iloc[j]
            fold = the_df['fold'].iloc[j]
            left =  min(the_df['sstart'].iloc[j], the_df['send'].iloc[j])
            right =  max(the_df['sstart'].iloc[j], the_df['send'].iloc[j]) + 1
            
            subsequence1_ref = extract_subsequence(ref_seq, left, right)
            subsequence2_ref = extract_subsequence(ref_seq, left, right, len_total=20000)
            ref_f1.write('%s,%s,%f,%i\n' % (the_id, subsequence1_ref, label, fold))
            ref_f2.write('%s,%s,%f,%i\n' % (the_id, subsequence2_ref, label, fold))
        

chr1 22372
chr2 19182
chr3 11968
chr4 8569
chr5 9141
chr6 13433
chr7 11130
chr8 12474
chr9 7554
chr10 9106
chr11 18154
chr12 9668
chr13 3415
chr14 5237
chr15 6343
chr16 10749
chr17 9123
chr18 3438
chr19 8127
chr20 9350
chr21 2670
chr22 4258


### extract mapped sequences with/without flank (length = ~200 or 20kb)

In [20]:
id_all = []
subsequence1_all = []
subsequence2_all = []

with open(output1_path, "w") as f1,  open(output2_path, "w") as f2:
    for i in range(1,23):
        target_chr = 'chr' + str(i)
        the_df = df0[df0.sseqid == str(i)]
        print('chr' + str(i), len(the_df))
        snp_probability_matrix = sparse.load_npz(path0 + 'snp_prob_' + target_chr + '.npz')
        ## encoded sequence for a chromosome
        encoded_seq = sample_variant(
            snp_probability_matrix,
            nucleotide_lexicon_path,
            biallele_lexicon_path,
            replacement=False,
        )
        
        for j in range(len(the_df)):
            the_id = the_df['qseqid'].iloc[j]
            label = the_df['mean_value'].iloc[j]
            fold = the_df['fold'].iloc[j]
            left =  min(the_df['sstart'].iloc[j], the_df['send'].iloc[j])
            right =  max(the_df['sstart'].iloc[j], the_df['send'].iloc[j]) + 1
            subsequence1 = extract_subsequence(encoded_seq, left, right)
            subsequence2 = extract_subsequence(encoded_seq, left, right, len_total=20000)
            f1.write(f'{the_id},{subsequence1},{label},{fold}\n')
            f2.write(f'{the_id},{subsequence2},{label},{fold}\n')


chr1 22372


 68%|██████▊   | 169245338/249250621 [24:11<11:25, 116627.29it/s]


KeyboardInterrupt: 

### calculate percentage of variations

In [8]:
var_percent1 = [count_variation_percent(x) for x in subsequence1_all]
print("percentage of variation(200): %.2f%%" % (sum(var_percent1)/len(var_percent1)*100))

percentage of variation(200): 0.75%


In [9]:
var_percent2 = [count_variation_percent(x) for x in subsequence2_all]
print("percentage of variation(20kb): %.2f%%" % (sum(var_percent2)/len(var_percent2)*100))

percentage of variation(20kb): 0.94%


### Now to create the train, test and dev data

In [ ]:
from pathlib import Path


def create_splits(output_dir, input_file):
    output_dir.mkdir(parents=True, exist_ok=True)

    df0 = pd.read_csv(input_file,sep=',', header=0)
    df0 = df0.astype({"fold":'int16'})
    print(df0.columns, df0.dtypes)

    train_df = df0.loc[df0['fold'].isin(range(6)),:]
    print(train_df.columns)
    print(train_df.shape, train_df.head(1))
    train_df[['chunk','mean_value']].to_csv(Path(output_dir / "train.csv"), index=False)

    test_df = df0.loc[df0['fold'].isin([8,9]),:]
    print(test_df.columns)
    print(test_df.shape, test_df.head(1))
    test_df[['chunk','mean_value']].to_csv(Path(output_dir / "test.csv"), index=False)

    dev_df = df0.loc[df0['fold'].isin([6,7]),:]
    print(dev_df.columns)
    print(dev_df.shape, dev_df.head(1))
    dev_df[['chunk','mean_value']].to_csv(Path(output_dir / "dev.csv"), index=False)

input_file = output1_ref_path
base_path = Path('/dccstor/bmfm-targets/data/omics/genome/MPGA/human_mpra')
output_dir = base_path / 'K562_ref_sequence_200'

create_splits(output_dir, input_file)
create_splits(base_path / 'K562_ref_sequence_20kb', output2_ref_path)
create_splits(base_path / 'K562_biallele_sequence_200', output1_path)
create_splits(base_path / 'K562_biallele_sequence_200', output2_path)


In [ ]:
df0['fold'].value_counts()